In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
import nltk
import numpy as np
import pandas as pd
import gensim
import transformers
nltk.download('stopwords')
from nltk.corpus import stopwords
sw_nltk = stopwords.words('english')
from gensim.models import Word2Vec, word2vec
from transformers import BertTokenizer, AutoTokenizer, BertModel, BertConfig, AutoModel, AdamW
import natsort
import os
from PIL import Image 
import time

[nltk_data] Downloading package stopwords to /home/nitesh/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
2023-07-15 00:10:06.892138: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [2]:
# Load the data
train2 = pd.read_csv('training1.csv', delimiter='\t')

IDs = []  #set1
images = []
directory = 'TRAINING'   # directory where we have images 
filenames = natsort.natsorted(os.listdir(directory))  

# get the ids from the images, where images are having three channels; omit images if channels != 3
for i, filename in enumerate(filenames):
#     print(i, filename)
    if filename.endswith(".jpg"):
#         ID = int(filename[:-4])
        ID = filename
        pathname = os.path.join(directory, filename)
        im = Image.open(pathname)
        im = im.resize((224, 224))  # Resize the image to (224, 224)
        imnp = np.array(im)
        if len(imnp.shape) != 3:
#             print("This is 1 channel, so we omit it", imnp.shape, filename)
            continue
        IDs.append(ID)
        images.append(imnp)
        

        
def get_common_strings(list1, list2):
    return list(set(list1) & set(list2))

# Example usage
list1 = IDs
list2 = list(train2.file_name)  #from the text file where we have text description 
common_strings = get_common_strings(list1, list2)
print('len of common strings', len(common_strings))



# Sort the dataframe with natural ordering of the IDs
train2['prefix_file_name'] = train2['file_name'].str.extract('(\d+)').astype(int)
# Assuming 'df' is your DataFrame
sorted_df = train2.sort_values(by='prefix_file_name', ascending=True)

# Print the sorted DataFrame
# print('sorted_df shape', sorted_df.shape)


# Assuming 'df' is your DataFrame and 'common_strings' is the list of strings
# Get the common string values in the column 
filtered_df = sorted_df[sorted_df['file_name'].isin(common_strings)].reset_index(drop=True)

# Print the filtered DataFrame
print('filtered_df shape', filtered_df.shape)

train3 = filtered_df.copy()
print(train3.shape)


# # We are just doing classification so no filename is required for our task. 
# train.drop(['file_name'], axis=1, inplace=True)
# train = train.iloc[:, [-1] + list(range(len(train.columns)-1))]
train = train3.rename(columns={'Text Transcription': 'text'})
display(train)

len of common strings 9986
filtered_df shape (9986, 8)
(9986, 8)


,file_name,misogynous,shaming,stereotype,objectification,violence,text,prefix_file_name
0,1.jpg,0,0,0,0,0,Milk Milk.zip,1
1,2.jpg,0,0,0,0,0,-What are you doing? -you told me to satanize ...,2
2,3.jpg,0,0,0,0,0,imgflip.com ME 1254 NEW BUGS AFTER CHANGES BUG...,3
3,4.jpg,0,0,0,0,0,Bedroom Kitchen Bathroom Bron memes storage,4
4,5.jpg,0,0,0,0,0,WAKEUP EARLY FREELANCERS,5
...,...,...,...,...,...,...,...,...
9981,15002.jpg,0,0,0,0,0,WAITING FOR THE END OF THE COVID imgflip.com,15002
9982,15003.jpg,0,0,0,0,0,SMART WOMEN ARE AROUND imgflip.com,15003
9983,15004.jpg,0,0,0,0,0,GOOD GIRLS ARE BEHIND THE CORNER imgflip.com,15004
9984,15005.jpg,0,0,0,0,0,COOKING FOR MY WIFE imgflip.com,15005


In [3]:
test1 = pd.read_csv('Test.csv', delimiter='\t')
test_labels = pd.read_csv('test_labels.txt', 
                          delimiter='\t',
                         header=None)

test_labels.columns = ['file_name', 
                      "misogynous",
                       "shaming",
                       "stereotype",
                       "objectification",
                       "violence"]

test2 = pd.merge(test1, test_labels, on='file_name', how='inner')

# train.drop(['file_name'], axis=1, inplace=True)
test2.drop(['file_name'], axis=1, inplace=True)
# train = train.iloc[:, [-1] + list(range(len(train.columns)-1))]


# train = train.rename(columns={'Text Transcription': 'text'})
test = test2.rename(columns={'Text Transcription': 'text'})
test

,text,misogynous,shaming,stereotype,objectification,violence
0,FACEBOOK SINGLES GROUPS BELIKE WHEN A NEW WOMA...,0,0,0,0,0
1,"SO, IF YOU'RE A FEMINIST HOW CAN YOU EAT DAIRY?",1,0,1,1,0
2,WHEN A CUTE GIRL LEFT YOUR MESSAGE ON SEEN,0,0,0,0,0
3,Photographing something you want to show every...,1,0,1,1,0
4,HEY BABE CAN YOU MAKE ME A SANDWICH? Hey babe ...,0,0,0,0,0
...,...,...,...,...,...,...
995,IT'S NOT YOUR FAULT You didn't design the dres...,1,0,1,1,0
996,THINK ABOUT HOW MUCH BETTER HER SKIN IS BREATH...,0,0,0,0,0
997,THE STEREOTYPES ARE TRUE F SHE DOES HAVE A TIG...,1,0,1,1,0
998,DRAWS NAKED PICTURES OF BLACK WOMEN 00 0000 GE...,0,0,0,0,0


In [4]:
def export_classification_report(report, mod, exec_time):
    lines = report.split('\n')
    data = lines[2:4] + lines[5:9]
    data = [line.split() for line in data]
    

    acc = float(data[2][1])
    m_prec = float(data[3][2])
    m_recall = float(data[3][3])
    m_f1 = float(data[3][4])
    w_prec = float(data[4][2])
    w_recall = float(data[4][3])
    w_f1 = float(data[4][4])
    
    df = pd.DataFrame({
        'Model': [mod],
        'Accuracy': [acc],
        'M-Precision': [m_prec],
        'M-Recall': [m_recall],
        'M-F1-Score': [m_f1],
        'W-Precision': [w_prec],
        'W-Recall': [w_recall],
        'W-F1-Score': [w_f1],
        'Runtime': [exec_time]
    })
    
    return df

In [5]:
import nltk
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
import re
import sys
import warnings
if not sys.warnoptions:
    warnings.simplefilter("ignore")

def remove_space(text):
    '''Removes awkward spaces'''
    text = text.strip()
    text = text.split()
    return " ".join(text)

def cleanPunc(text): 
    '''function to clean the word of any punctuation or special characters'''
    cleaned = re.sub(r'[?|!|\'|"|#]',r'',text)
    cleaned = re.sub(r'[.|,|)|(|\|/]',r' ',cleaned)
    cleaned = cleaned.strip()
    cleaned = cleaned.replace("\n"," ")
    return cleaned

def keepAlpha(text):
    alpha_sent = ""
    for word in text.split():
        alpha_word = re.sub('[^a-z A-Z]+', ' ', word)
        alpha_sent += alpha_word
        alpha_sent += " "
    alpha_sent = alpha_sent.strip()
    return alpha_sent

def text_preprocessing_pipeline(text):
    '''Cleaning and parsing the text.'''
    text = remove_space(text)
    text = cleanPunc(text)
    text = keepAlpha(text)
    return text

train['clean_text'] = train['text'].str.lower()
train['clean_text'] = train['text'].apply(text_preprocessing_pipeline)
test['clean_text'] = test['text'].apply(text_preprocessing_pipeline)
train.head()


stemmer = SnowballStemmer("english")
def stemming(sentence):
    stemSentence = ""
    for word in sentence.split():
        
        stem = stemmer.stem(word)
        stemSentence += stem
        stemSentence += " "
    stemSentence = stemSentence.strip()
    return stemSentence

train['clean_text'] = train['clean_text'].apply(stemming)
test['clean_text'] = test['clean_text'].apply(stemming)


from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer(stop_words="english")

# create TF-IDF features
xtrain_tfidf = tfidf_vectorizer.fit_transform(train['clean_text'])
xtest_tfidf = tfidf_vectorizer.transform(test['clean_text'])

In [6]:
import random

# Prepare the data
X_train = xtrain_tfidf
y_train = train['misogynous']


X_test = xtest_tfidf
y_test = test['misogynous']

# np.random.seed(7)
# bootstrap_iterations=5
df=pd.DataFrame(columns=['Model', 'Accuracy', 
                       'M-Precision', 'M-Recall', 
                       'M-F1-Score', 'W-Precision',
                       'W-Recall', 'W-F1-Score', 'Runtime'])

# for _ in range(bootstrap_iterations):
    
for i in list([7,10,18,22,55,77]):
    
    random.seed(i)
    np.random.seed(i)
#     torch.manual_seed(i)
#     tf.random.set_seed(i)
    
    start_time = time.time()
        
    # Perform bootstrap resampling on the training data
    bootstrap_indices = np.random.choice(X_train.shape[0], size=X_train.shape[0], replace=True)
    X_train_bootstrap = X_train[bootstrap_indices]
    y_train_bootstrap = y_train[bootstrap_indices]

    # Train a Naive Bayes classifier
    nb = MultinomialNB()
    nb.fit(X_train_bootstrap, y_train_bootstrap)

    # Predict on test data
    y_pred = nb.predict(X_test)
    report = classification_report(list(y_test), list(y_pred), digits=3) 
#     print(report)
    end_time = time.time()
    execution_time1 = end_time - start_time

    print(f"Execution time: {execution_time1} seconds")
    model= 'NB'
    df1 = export_classification_report(report, model,  execution_time1)
#     print(df)
    df = pd.concat([df, df1])
display(df)





Execution time: 0.00857400894165039 seconds
Execution time: 0.0047528743743896484 seconds
Execution time: 0.004613637924194336 seconds
Execution time: 0.004480123519897461 seconds
Execution time: 0.004380702972412109 seconds
Execution time: 0.00436854362487793 seconds


,Model,Accuracy,M-Precision,M-Recall,M-F1-Score,W-Precision,W-Recall,W-F1-Score,Runtime
0,NB,0.590,0.602,0.590,0.578,0.602,0.590,0.578,0.008574
0,NB,0.601,0.617,0.601,0.587,0.617,0.601,0.587,0.004753
0,NB,0.592,0.607,0.592,0.577,0.607,0.592,0.577,0.004614
0,NB,0.602,0.618,0.602,0.588,0.618,0.602,0.588,0.004480
0,NB,0.594,0.607,0.594,0.581,0.607,0.594,0.581,0.004381
0,NB,0.590,0.605,0.590,0.575,0.605,0.590,0.575,0.004369


In [7]:
df.to_excel('1_classification_report_NB.xlsx', index=False)

In [8]:
# with pd.ExcelWriter("classification_report.xlsx") as writer:
#     df.to_excel(writer, sheet_name="Sheet1") 

In [9]:
# # import pandas as pd
# # from openpyxl import load_workbook

# # # Load the existing Excel file
# # excel_file = 'classification_report.xlsx'
# # existing_data = pd.read_excel(excel_file)

# # # Append new dataframe rows
# # new_data = pd.concat([existing_data, df], ignore_index=True)

# !pip install deepcopy